In [50]:
%reload_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')

from pathlib import Path
import numpy as np
from scipy import stats
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Increase the resolution of all the plots below
plt.rcParams.update({"figure.dpi": 300,"figure.facecolor":"w","figure.figsize": (15,10)})

from src.utl import remove_outliers_df, nan_pearsonr, remove_outliers
from src.config import dir_rawdata, dir_prep, dir_plots_group_cmpr, cfg_colors


In [10]:
# set colors for plotting 
cc_low = cfg_colors["group_colors"][0]
cc_high = cfg_colors["group_colors"][1]

## set path and merge data

In [11]:
dat_pupil = pd.read_csv(Path.joinpath(dir_prep,"all_trials_pupil.csv"), index_col=False)
dat_fsr = pd.read_csv(Path.joinpath(dir_prep,"all_trials_fsr.csv"), index_col=False)
dat_clin = pd.read_excel(Path.joinpath(dir_rawdata,"behav_data.xlsx"), index_col=False)

physio = pd.merge(dat_fsr,dat_pupil)

In [12]:
# clean data
all = dat_clin[dat_clin["ID"].str.contains("p2|c3")]
all["group"] = all.ID.str[0]


In [13]:
full_set = all.merge(physio)
idx_pat = full_set.group == "p"

pat = full_set[idx_pat]
hc = full_set[~idx_pat]

In [14]:
print(pat.columns)

Index(['ID', 'Alter', 'Geschlecht', 'Krankheitsdauer', 'BDI-Score',
       'Schahmann semantische Wortflüssigkeit',
       'Schahmann phonematische Wortflüssigkeit',
       'Schahmann Kategorie-Wechsel', 'Schahmann Zahlen-spanne vorwärts',
       'Schahmann Zahlen-spanne rückwärts', 'Schahmann Würfel ',
       'Schahmann verbaler Abruf', 'Schahmann Gemeinsam-keiten',
       'Schahmann Go/No-Go', 'Schahmann Affekt',
       'Schahmann Gesamtwert Aufgaben', 'Schahmann Gesamtwert Module',
       'Tetras Sprechen', 'Tetras mit dem Löffel essen',
       'Tetras aus einem Glas trinken', 'Tetras Körper-hygiene',
       'Tetras Ankleiden', 'Tetras Schütten/Ein-schenken',
       'Tetras Tablett tragen', 'Tetras Schlüssel benutzen',
       'Tetras Schreiben', 'Tetras Arbeiten',
       'Tetras Tätigkeit mit schwerster Beeinträchti-gung',
       'Tetras Grad der Beeinträchti-gung ',
       'Tetras soziale Beeinträchti-gung', 'Tetras Gesamtscore Interview',
       'Tetras Kopftremor', 'Tetras Gesich

In [15]:
diffs = pat.pivot_table(index=['ID','Group','Feedback type'],columns='Feedback angle',values=['Power [4-12]','Pupil size'],margins=False,aggfunc='mean').reset_index()
diffs['dPower'] = diffs['Power [4-12]']['high']-diffs['Power [4-12]']['low']
diffs['dPpl'] = diffs['Pupil size']['high']-diffs['Pupil size']['low']
diffs['dPower'] = abs(diffs['dPower'])

In [16]:
print(diffs.columns)

MultiIndex([(           'ID',     ''),
            (        'Group',     ''),
            ('Feedback type',     ''),
            ( 'Power [4-12]', 'high'),
            ( 'Power [4-12]',  'low'),
            (   'Pupil size', 'high'),
            (   'Pupil size',  'low'),
            (       'dPower',     ''),
            (         'dPpl',     '')],
           names=[None, 'Feedback angle'])


In [17]:
pat.head()

,ID,Alter,Geschlecht,Krankheitsdauer,BDI-Score,Schahmann semantische Wortflüssigkeit,Schahmann phonematische Wortflüssigkeit,Schahmann Kategorie-Wechsel,Schahmann Zahlen-spanne vorwärts,Schahmann Zahlen-spanne rückwärts,...,Gesamtscore Tetras,group,Trial n,Feedback type,Feedback angle,Power [4-12],Power [1-3],Group,Pupil size,Percentage bad pupil samples
0,p214,NaN,NaN,NaN,NaN,24,15,13,6,5,...,NaN,p,0,vo,low,1.270050e-07,0.000053,p,0.325861,1.409091
1,p214,NaN,NaN,NaN,NaN,24,15,13,6,5,...,NaN,p,1,vo,low,2.389101e-07,0.000054,p,0.368568,1.818182
2,p214,NaN,NaN,NaN,NaN,24,15,13,6,5,...,NaN,p,2,vo,high,6.601151e-08,0.000056,p,0.182429,1.818182
3,p214,NaN,NaN,NaN,NaN,24,15,13,6,5,...,NaN,p,3,vo,high,1.076244e-07,0.000056,p,0.098450,2.590909
4,p214,NaN,NaN,NaN,NaN,24,15,13,6,5,...,NaN,p,4,va,low,1.312593e-07,0.000052,p,0.137279,3.590909


## check Tremor power vs. TETRAS 

In [51]:

# calculate the mean power per feedback condition for low angle
pow_low_vo = remove_outliers(pat[np.logical_and(pat["Feedback angle"] == 'low', pat["Feedback type"] == 'vo')].groupby("ID")["Power [4-12]"].mean(),1)
pow_low_va = remove_outliers(pat[np.logical_and(pat["Feedback angle"] == 'low', pat["Feedback type"] == 'va')].groupby("ID")["Power [4-12]"].mean(),1)
pow_low_ao = remove_outliers(pat[np.logical_and(pat["Feedback angle"] == 'low', pat["Feedback type"] == 'ao')].groupby("ID")["Power [4-12]"].mean(),1)

# calculate the mean power per feedback condition as difference between angles
pow_diff_vo =  pat[np.logical_and(pat["Feedback angle"] == 'high', pat["Feedback type"] == 'vo')].groupby("ID")["Power [4-12]"].mean() - pow_low_vo
pow_diff_va =  pat[np.logical_and(pat["Feedback angle"] == 'high', pat["Feedback type"] == 'va')].groupby("ID")["Power [4-12]"].mean() - pow_low_va
pow_diff_ao =  pat[np.logical_and(pat["Feedback angle"] == 'high', pat["Feedback type"] == 'ao')].groupby("ID")["Power [4-12]"].mean() - pow_low_ao

pow_vars = [pow_low_vo, pow_low_va, pow_low_ao, pow_diff_vo, pow_diff_va, pow_diff_ao]


# Tetras total score per patient
tetras = pat.groupby("ID")["Gesamtscore Tetras"].mean()

# TO DO:All TETRAS OE items (TETRAS Hand Score)

In [52]:
vars_tetras_hand = ['Tetras Tremor OE ausgestreckt RE', 'Tetras Tremor OE ausgestreckt LI',
'Tetras Tremor OE Flügelschlag RE', 'Tetras Tremor OE Flügelschlag LI',
'Tetras Tremor OE kinet. Tremor RE',
'Tetras Tremor OE kinet. Tremor LI',]

# Tetras total score per patient
hand_tetras = pat.groupby("ID")[vars_tetras_hand].mean()
hand_tetras = hand_tetras.sum(axis=1)

In [62]:
r, p = nan_pearsonr(pow_low_vo, hand_tetras)
print(f"The correlation of Tremor Power in visual only, low feedback and TETRAS Hand Score is r = {r:.4f}, p = {p:.2f}")
r, p = nan_pearsonr(pow_low_va, hand_tetras)
print(f"The correlation of Tremor Power in auditiv-visual, low feedback and TETRAS Hand Score is r = {r:.4f}, p = {p:.2f}")
r, p = nan_pearsonr(pow_low_ao, hand_tetras)
print(f"The correlation of Tremor Power in auditiv only, low feedback and TETRAS Hand Score is r = {r:.4f}, p = {p:.2f}")

r, p = nan_pearsonr(pow_diff_ao, hand_tetras)
print(f"The correlation of Tremor Power in auditiv only feedback, low-high and TETRAS Hand Score is r = {r:.4f}, p = {p:.2f}")


The correlation of Tremor Power in visual only, low feedback and TETRAS Hand Score is r = 0.3506, p = 0.18
The correlation of Tremor Power in auditiv-visual, low feedback and TETRAS Hand Score is r = 0.3790, p = 0.15
The correlation of Tremor Power in auditiv only, low feedback and TETRAS Hand Score is r = 0.6020, p = 0.01
The correlation of Tremor Power in auditiv only feedback, low-high and TETRAS Hand Score is r = 0.3286, p = 0.21


## check Tremor pupil vs. TETRAS 

In [28]:

# calculate the mean power per feedback condition for low angle
pup_low_vo = pat[np.logical_and(pat["Feedback angle"] == 'low', pat["Feedback type"] == 'vo')].groupby("ID")["Pupil size"].mean()
pup_low_va = pat[np.logical_and(pat["Feedback angle"] == 'low', pat["Feedback type"] == 'va')].groupby("ID")["Pupil size"].mean()
pup_low_ao = pat[np.logical_and(pat["Feedback angle"] == 'low', pat["Feedback type"] == 'ao')].groupby("ID")["Pupil size"].mean()

# calculate the mean power per feedback condition as difference between angles
pup_diff_vo =  pat[np.logical_and(pat["Feedback angle"] == 'high', pat["Feedback type"] == 'vo')].groupby("ID")["Pupil size"].mean() - pow_low_vo
pup_diff_va =  pat[np.logical_and(pat["Feedback angle"] == 'high', pat["Feedback type"] == 'va')].groupby("ID")["Pupil size"].mean() - pow_low_va
pup_diff_ao =  pat[np.logical_and(pat["Feedback angle"] == 'high', pat["Feedback type"] == 'ao')].groupby("ID")["Pupil size"].mean() - pow_low_ao

pup_vars = [pup_low_vo, pup_low_va, pup_low_ao, pup_diff_vo, pup_diff_va, pup_diff_ao]

# Tetras total score per patient
tetras = pat.groupby("ID")["Gesamtscore Tetras"].mean()
# TO DO: All TETRAS OE items (TETRAS Hand Score)

In [60]:
r, p = nan_pearsonr(pup_low_vo, hand_tetras)
print(f"The correlation of Pupil Size in visual only, low feedback and TETRAS Hand Score is r = {r:.4f}, p = {p:.2f}")
r, p = nan_pearsonr(pup_low_va, hand_tetras)
print(f"The correlation of Pupil Size in auditiv-visual, low feedback and TETRAS Hand Score is r = {r:.4f}, p = {p:.2f}")
r, p = nan_pearsonr(pup_low_ao, hand_tetras)
print(f"The correlation of Pupil Size in auditiv only, low feedback and TETRAS Hand Score is r = {r:.4f}, p = {p:.2f}")



The correlation of Pupil Size in visual only, low feedback and TETRAS Hand Score is r = -0.2431, p = 0.36
The correlation of Pupil Size in auditiv-visual, low feedback and TETRAS Hand Score is r = 0.0319, p = 0.91
The correlation of Pupil Size in auditiv only, low feedback and TETRAS Hand Score is r = 0.1323, p = 0.63


## check pupil tremor low condition

In [31]:
pup_vars = [pup_low_vo, pup_low_va, pup_low_ao]
pow_vars = [pow_low_vo, pow_low_va, pow_low_ao]

for vp, vt in zip(pup_vars ,pow_vars):
    r, p = nan_pearsonr(vp, vt)

    print(f"r = {r:.2f}, p = {p:.2f}")

r = -0.03, p = 0.91
r = 0.24, p = 0.37
r = -0.13, p = 0.62
